In [2]:
import re
import random

def clean(s):
    regPattern = ['(^[0-9]+$)', '^[A-Z]+$', '^[A-Z]+\.$']
    regObj = [re.compile(r) for r in regPattern]    
    
    ret = []
    s = s.strip().split()
    for w in s:
        w = w.strip()
        rTests = [(r.match(w) == None) for r in regObj]
        if False not in rTests:
            ret.append(w)
    return ret

# clean("123 123. foo bar. BAR. 1 ABC")

def sepTriple(((k),v)):
    print k, v
    return ((k[0], k[1]), [(k[2],v)])

print 'Cleaning input...'

allWords = []
with open('pg100.txt','r') as f:
    for line in f:
        allWords += clean(line)

print 'Creating RDD...'
        
rddList = []
numP = 2
forW = 3

for i in xrange(0, len(allWords)-forW):
    rddList.append( tuple(allWords[i:i+forW]) )

RDD = sc.parallelize(rddList, numP)

RDD = RDD.map( lambda (v): (v, 1) )
RDD = RDD.reduceByKey(lambda a, b: a+b)
RDD = RDD.map( sepTriple )\
            .reduceByKey( lambda a,b: a+b)\
            .cache()

def randPick(words):
    norm = 0.    
    r = random.uniform(0.,1.)
    for k,v in words:
        norm += v        
    for k,v in words:
        r -= float(v) / norm
        if r < 0:
            return k       
    return "ERROR"

# indexed = RDD.zipWithIndex()
# print indexed.take(10)

print 'Generating random words...'

for i in xrange(10):
    phrase = []
    first = RDD.takeSample(True, 1, None)[0]
    phrase += list(first[0])
    print 'Starting with', phrase
    for j in xrange(18):
        look = RDD.filter(lambda ( (k), v ): (k == tuple(phrase[-2:]))).take(1)

        if len(look) != 1:
            print "ERROR: no results returned, selecting random word"
            look = RDD.take(1) # Take a random word
        ((k),v) = look[0]
        phrase.append(randPick(v))
        
    print ' '.join(phrase)

Cleaning input...
Creating RDD...
Generating random words...
Starting with ['the', 'chantry']
the chantry by; there, before the last, not least in love, Upbraid my falsehood when th' hast wrong'd me. My
Starting with ['the', 'pith']
the pith of business 'Twixt you there's difference; but the most unnatural and faithless service. Heaven has an excellent good
Starting with ['answers,', 'And']
answers, And poise the cause why we are not still keep her ignorant of what you will. An may hide
Starting with ['desires', 'Buys']
desires Buys herself bread and clothes. It is Lord Angelo dukes it well befits you should tender your supposed aid,
Starting with ['maids;', 'so']
maids; so without these follies are within the hollow mine of gold. Other, less fine in love, or no, must
Starting with ['My', 'lungs']
My lungs began to reign, Before whose glory was great of birth; And that, my liege, my wife a handkerchief-
Starting with ['Either', 'a']
Either a coward than a fellow, look you there. But, s